In [81]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from other_repos.pyDMS.pyDMS.pyDMS import *
import time
from joblib import Parallel, delayed

os.environ["GDAL_MAX_DATASET_POOL_SIZE"] = "600"


def runSharpi(highResFilename, lowResFilename, lowResMaskFilename, cv, movWin, regrat, outputFilename, useDecisionTree = True):
    commonOpts = {"highResFiles":               [highResFilename],
                    "lowResFiles":              [lowResFilename],
                    "lowResQualityFiles":         [lowResMaskFilename],
                    "lowResGoodQualityFlags":     [1],
                    "cvHomogeneityThreshold":     cv,
                    "movingWindowSize":           movWin,
                    "disaggregatingTemperature":  True}
    dtOpts =     {"perLeafLinearRegression":    True,
                    "linearRegressionExtrapolationRatio": round(regrat, 2)}
    sknnOpts =   {'hidden_layer_sizes':         (10,),
                    'activation':                 'tanh'}
    nnOpts =     {"regressionType":             REG_sklearn_ann,
                    "regressorOpt":               sknnOpts}

    start_time = time.time()
    opts = commonOpts.copy()
    if useDecisionTree:
        opts.update(dtOpts)
        disaggregator = DecisionTreeSharpener(**opts)
    else:
        opts.update(nnOpts)
        disaggregator = NeuralNetworkSharpener(**opts)

    print("Training regressor...")
    disaggregator.trainSharpener()
    print("Sharpening...")
    downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
    print("Residual analysis...")
    residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                    lowResMaskFilename,
                                                                    doCorrection=True)
    print("Saving output...")

    if correctedImage is not None:
        outImage = correctedImage
    else:
        outImage = downscaledFile
    # outData = utils.binomialSmoother(outData)
    outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                            outImage.GetGeoTransform(),
                            outImage.GetProjection(),
                            f'{os.path.split(outputFilename)[0]}/Values/{os.path.split(outputFilename)[1]}')
    residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                residualImage.GetGeoTransform(),
                                residualImage.GetProjection(),
                                f'{os.path.split(outputFilename)[0]}/Residuals/{os.path.split(outputFilename)[1]}_resid{os.path.splitext(outputFilename)[1]}')

    outFile = None
    residualFile = None
    downsaceldFile = None
   
    print(time.time() - start_time, "seconds")

In [82]:
# paths
lowmask_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP.tif'
lowmask_bin_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP_BINARY.tif'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump3/'

# create vrts of slope, aspect and landcover (for masking)
path_to_slope = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/'
path_to_aspect = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/'
path_to_agro = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/THUENEN_2021/'

# get all highRes datasets: S2 composites, aspect, ratio and incedence for the tile/time and stack them (in a composite if more than one tile is processed)
# set the highRes S2 tiles that will be processed

# get the Tiles for Brandenburg
bran = pd.read_csv('/data/Aldhani/eoagritwin/misc/state_tile_csv/clipped_grid_bran_tiles.csv')
tiles_to_process = createFORCEtileLIST(list(bran['Tile_X']),
                                        list(bran['Tile_Y']), True)





In [7]:
years = [year for year in range(2017, 2025, 1)]
months = [f'{month:02d}' for month in range(1, 13, 1)]
dates = []
for year in years:
    for month in months:
        next_month = int(month) + 1
        if next_month > 12:
            next_month = 1
            next_year = year + 1
        else:
             next_year = year
        dates.append([f'{year}-{month}-01', f'{year}-{month}-15'])
        dates.append([f'{year}-{month}-15', f'{next_year}-{next_month:02d}-01'])

germany = {"west": 5.592041, "south": 47.129951, "east": 15.26001, "north": 55.09723}

dates = dates[85:87]

In [8]:
dates

[['2020-07-15', '2020-08-01'], ['2020-08-01', '2020-08-15']]

In [10]:
while dates:

    for date in dates[:]:  # Iterates over a copy of the list to allow removal at the end without messing up the order

        storPath = f"/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_{date[0]}.nc"
        print(storPath)

/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-08-01.nc
/data/Aldhani/eoagritwin/et/Sentinel3/raw_VAA_masked/Germany_2020-07-15.nc
/data/Aldhani/eoagritwin/

KeyboardInterrupt: 

In [ ]:
# # make a specific folder for this run and store the info together
# csv_path = f'{temp_dump}folders.csv'
# rand_foldname = getUniqueIDfromTILESXY(Tile_X, Tile_Y)
# temp_dump_fold = f'{temp_dump}{rand_foldname}/'

# if not os.path.exists(temp_dump_fold):
#     os.makedirs(temp_dump_fold,exist_ok=False)
    
#     df = pd.DataFrame({
#             'Folder': temp_dump_fold,
#             'Tile_X': Tile_X,
#             'Tile_Y': Tile_Y   
#         })

#     if not os.path.exists(csv_path):
#         df.to_csv(csv_path, index=False)
#     else:
#         df_exist = pd.read_csv(csv_path)
#         df_new = pd.concat([df_exist, df], ignore_index=True)
#         df_new.to_csv(csv_path, index=False)
#         # add the lines

# slope-tiles
slopes = [file for file in getFilelist(path_to_slope, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file
# aspect-tiles
aspects = [file for file in getFilelist(path_to_aspect, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file
# thuenen-tiles
thuenen = [file for file in getFilelist(path_to_agro, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file

# get those tiles (and composite if more than one tile is provided)
slope_path = f'{temp_dump_fold}SLOPE.vrt'
gdal.BuildVRT(slope_path, slopes)

aspect_path = f'{temp_dump_fold}ASPECT.vrt'
gdal.BuildVRT(aspect_path, aspects)

thuenen_path = f'{temp_dump_fold}THUENEN.vrt'
gdal.BuildVRT(thuenen_path, thuenen)

In [83]:
# year 
year = 2019
  
# paths
path_to_S2_tiles = f'/data/Aldhani/eoagritwin/force/output/Guzinski/{year}/'

##### which tiles should be processed
# get a list with all available tiles
files = getFilelist(f'{path_to_S2_tiles}/tiles', '.tif', deep=True) 
files = [file for file in files if any(tile in file for tile in tiles_to_process)]
date_list = check_forceTSI_compositionDates(files)

# th_ds = gdal.Open(thuenen_path)
# th_arr = th_ds.GetRasterBand(1).ReadAsArray()
# mask = np.where(th_arr == -9999, 0, 1)

lowRes_files = []
highRes_files = []
highRes_names = []

colors = ['BLU', 'GRN', 'RED', 'NIR', 'RE1', 'RE2', 'RE3',  'SW1', 'SW2']

all dates of composites are the same :)


In [ ]:
S2_path = f'{temp_dump_fold}S2_{date}.vrt'
vrt = gdal.BuildVRT(S2_path, tilesS2, separate=True)
vrt = None
vrt = gdal.Open(S2_path, gdal.GA_Update)  # VRT must be writable
for idz, bname in enumerate(colors): 
    band = vrt.GetRasterBand(1+idz)
    band.SetDescription(bname)
vrt = None

In [ ]:
arr_vrt = stackReader(S2_path)

In [88]:
dateList_for_csv = []

#### S2 composites are time sensitive (need to be aligned with date of LST observation), so is incidence
for date in date_list:

    # if not os.path.exists(f'{temp_dump_fold}INCIDENCE_{date}.vrt'):
    if int('20190401') <= int(date) <= int('20190801'): 
    

        dateList_for_csv.append(date)
        
df = pd.DataFrame({
    'compdates': dateList_for_csv
    })
df.to_csv('/data/Aldhani/eoagritwin/et/Sentinel3/LST/SharpEvap/Brandenburg/evap/compdates.csv', index=False)

    # # check if for that date vrts were already processed
    # if os.path.exists(f'{temp_dump_fold}S2_{date}'):
    #     print('S2 data for this date already processed')
    #     not_masked = [file for file in getFilelist(temp_dump_fold, '.vrt') if 'HIGHRES' in file]
    #     masked = [file for file in getFilelist(temp_dump_fold, '.tif') if 'HIGHRES' in file]
    #     highRes_files = [item for pair in zip(not_masked, masked) for item in pair]
    #     highRes_names = ["S2notMasked" if ".vrt" in file else "S2Masked" for file in highRes_files]
    #     lowRes_files = [f for f in [file for file in getFilelist(temp_dump_fold, '.tif') if 'Daily_LST' in file] for _ in range(2)]
    #     continue
    # else:
    #     # get those tiles (and composite if more than one tile is provided)
    #     if len(tiles_to_process) == 1:

    #         tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if tiles_to_process[0] in file and f'{date}.tif' in file]
    #         tilesS2 = [t2 for col in colors for t2 in tilesS2 if col in t2]
    #         S2_path = f'{temp_dump_fold}S2_{date}.vrt'
    #         vrt = gdal.BuildVRT(S2_path, tilesS2, separate=True)
    #         vrt = None
    #         vrt = gdal.Open(S2_path, gdal.GA_Update)  # VRT must be writable
    #         for idz, bname in enumerate(colors): 
    #             band = vrt.GetRasterBand(1+idz)
    #             band.SetDescription(bname)
    #         vrt = None

    #     else:
    #         tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and f'{date}.tif' in file] 
    #         force_to_vrt(tilesS2,
    #                 getCOLORSinOrderFORCELIST(tilesS2, colors, single=False),
    #                 f'{temp_dump_fold}S2_{date}',
    #                 False,
    #                 bandnames= list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)))
            
    #         S2_path = [file for file in getFilelist(f'{temp_dump_fold}S2_{date}', '.vrt', deep=True) if '_Cube' in file][0]
            
    #         # determine LST and incidence files associated with respective S2 composite
    #     band_dict = transform_compositeDate_into_LSTbands(date, 4)


    #     # stat used for compositing
    #     for comp_stat in ['minVZA', 'maxLST']:
    #         path_to_incident = f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/INCIDENCE/{comp_stat}/{year}/'
    #         path_to_LST = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/LST_composites/{comp_stat}/{year}/'

    #         # get all LST bands that can be sharped with the S2 composite at this date (and sun angle incidence files as well, as they are dependent on that date
    #         LSTs = []

    #         for k, v in band_dict.items():
    #             month = v['month']
    #             band = int(v['band'])
    #             v_path = f'{path_to_LST}Daily_LST_{comp_stat}_{year}_{month}.tif'
    #             ds = gdal.Open(v_path, 0)
                
    #             # export the LST for that day
    #             LST_arr = ds.GetRasterBand(band).ReadAsArray() # store as single Tiff in temp
    #             makeTif_np_to_matching_tif(LST_arr, v_path, f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')

    #             # store the paths for selecting incidence for corresponding LST
    #             incid_date = f'{year}_{month}_{band:02d}.tif'
    #             lowRes_files.append(f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')

    #             # incidence-tiles
    #             incids = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and incid_date in file] 
    #             # get those tiles (and composite if more than one tile is provided)
    #             if len(tiles_to_process) == 1:
    #                 incid_path = incids[0]

    #             else:
    #                 incid_path = f'{temp_dump_fold}INCIDENCE_{comp_stat}_{incid_date.split('.')[0]}.vrt'
    #                 gdal.BuildVRT(incid_path, incids)

    #             # create highRes file through exapnding the vrt of S2
    #             highRes_path = f'{temp_dump_fold}HIGHRES_{comp_stat}_{incid_date.split('.')[0]}.vrt'
    #             gdal.BuildVRT(highRes_path, [S2_path, slope_path, aspect_path, incid_path], separate=True)
    #             maskVRT_water(highRes_path)
    #             highRes_files.append(f'{highRes_path.split('.')[0]}_watermask.tif')
    #             highRes_names.append('S2notMasked')
    #             maskVRT(f'{highRes_path.split('.')[0]}_watermask.tif', mask)
    #             highRes_files.append(f'{highRes_path.split('.')[0]}_watermask_S2_agromask.tif')
    #             lowRes_files.append(f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')
    #             highRes_names.append('S2Masked')

In [ ]:
date_list

In [ ]:
joblist = []
outFolder = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/{rand_foldname}/'
for idx, highResFilename in enumerate(highRes_files):
    lowResFilename = lowRes_files[idx]
    f1 = f'{outFolder}{'/'.join(highResFilename.split('.')[0].split('_')[2:6])}/'
    for maskname, mask_lowRes in zip(['withoutLSTmask', 'withLSTmask'], ['', lowmask_bin_path]):
        lowResMaskFilename = mask_lowRes
        f2 = f'{f1}{maskname}/'
        for movWin in [15]:
            for cv in [0]:
                for regrat in [0.25]:
                    kombi = f'mvwin{movWin}_cv{cv}_regrat{int(regrat*100):02d}_{highRes_names[idx]}_{maskname}'
                    f3 = f'{f2}{highRes_names[idx]}/'
                    os.makedirs(f'{f3}Residuals/', exist_ok=True)
                    os.makedirs(f'{f3}Values/', exist_ok=True)
                    joblist.append([highResFilename, 
                                lowResFilename,
                                lowResMaskFilename,
                                cv, movWin, regrat,
                                f'{f3}{'_'.join(highResFilename.split('.')[0].split('_')[2:6])}_{kombi}.tif'])

print(f'\n{len(joblist)} times will be sharpened\n')

In [ ]:
if __name__ == '__main__':
    starttime = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
    print("--------------------------------------------------------")
    print("Starting process, time:" + starttime)
    print("")
   
    Parallel(n_jobs=20)(delayed(runSharpi)(job[0], job[1], job[2], job[3], job[4], job[5], job[6]) for job in joblist)


    print("")
    endtime = time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
    print("--------------------------------------------------------")
    print("--------------------------------------------------------")
    print("start : " + starttime)
    print("end: " + endtime)
    print("")


In [ ]:
!jupyter nbconvert --to script sharper_test.ipynb --output /home/potzschf/repos/evapo_scripts/Guzinski/py/sharper_test_in_PARALLEL_14_09

In [ ]:
aa = np.ones((3,4,23))
print(aa.shape)

bb = aa[:,:,0:9]
print(bb.shape)